In [52]:
import onnx
from onnx import helper,TensorProto
from finn.core.modelwrapper import ModelWrapper
from finn.core.datatype import DataType
import numpy as np
from finn.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

from finn.transformation.infer_shapes import InferShapes

model = ModelWrapper("tinyyolo-20210831.onnx")
model.get_initializer("737")


array([0])

In [53]:
# shape node 
top_in = helper.make_tensor_value_info("top_in", TensorProto.FLOAT, [1,64,16,16])
shape_out = helper.make_tensor_value_info("735", TensorProto.INT64,[4])
shape_node = helper.make_node(
            "Shape",
            inputs = ["top_in"],
            outputs = ["735"])
shape_graph = helper.make_graph(
                [shape_node],
                inputs = [top_in],
                outputs = [shape_out],
                name = "shape_graph"
                )
shape_model = ModelWrapper(helper.make_model(shape_graph))
shape_model.transform(InferShapes())
shape_model.save("Shape_model.onnx")

In [80]:
# Slice node
input_shape = [4]
output_shape = [2]
param_shape = [1]
idt = DataType["UINT8"]
param_dt = DataType["INT64"]
slice_in = helper.make_tensor_value_info("735", TensorProto.INT64, input_shape)
slice_out = helper.make_tensor_value_info("slice_out", TensorProto.INT64, output_shape)
slice_attr = {}

value_info = [
        helper.make_tensor_value_info("starts", TensorProto.INT8, param_shape),
        helper.make_tensor_value_info("ends",   TensorProto.INT8, param_shape),
        helper.make_tensor_value_info("axes",   TensorProto.INT8, param_shape),
]

slice_node = helper.make_node(
            "Slice",
            inputs  = ["735","starts","ends","axes"],
            outputs = ["slice_out"],
            **slice_attr
            )

slice_graph = helper.make_graph(
                [slice_node],
                inputs = [slice_in],
                outputs = [slice_out],
                name = "slice_graph"
                )
slice_model = ModelWrapper(helper.make_model(slice_graph))

slice_model.set_initializer("starts",np.array([1]))
slice_model.set_initializer("ends",np.array([1]))
slice_model.set_initializer("axes",np.array([1]))

slice_model.transform(InferShapes())
slice_model.save("Slice_model.onnx")

In [81]:
# Concat
concat_in = helper.make_tensor_value_info("slice_out", TensorProto.INT64, [2])
p1 = helper.make_tensor_value_info("839", TensorProto.INT64,[2])
top_out = helper.make_tensor_value_info("top_out", TensorProto.INT64,[4])
concat_config = {}
concat_config["axis"] = np.int64(0)
concat_node = helper.make_node(
            "Concat",
            inputs = ["slice_out","839"],
            outputs = ["top_out"],
            **concat_config)

concat_model = helper.make_model(
    
    helper.make_graph(    
        [concat_node],
        inputs=[concat_in],
        outputs = [top_out],
        name = "concat_graph",
        value_info=[p1])
)

Concat_model = ModelWrapper(concat_model)
Concat_model.set_initializer("839",np.array([10,10]))
Concat_model.transform(InferShapes())
Concat_model.save("Concat_model.onnx")

In [82]:
# Create graph
graph = helper.make_graph(
    nodes = [shape_node,slice_node,concat_node],
    name = "slice_graph",
    inputs = [top_in],
    outputs = [top_out],
    value_info = value_info,
)

In [83]:
# Create full model
model_config = {}
model_config["opset_imports"] = [helper.make_operatorsetid("",11)]

modelProto = helper.make_model(graph,**model_config)
model = ModelWrapper(modelProto)

model.set_initializer("starts",np.array([4]))
model.set_initializer("ends",np.array([3]))
model.set_initializer("axes",np.array([2]))
model.set_initializer("839",np.array([10,10]))

model.transform(InferShapes())
model.save("slice.onnx")

In [85]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "slice.onnx"

final_output_dir = "output_files/slice"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
#     steps               = "estimate_only_dataflow_steps",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

In [86]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 196, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/finn-base/src/finn/core/modelwrapper.py", line 141, in transform
    (transformed_model, model_was_changed) = transformation.apply(
  File "/workspace/finn-base/src/finn/transformation/fold_constants.py", line 54, in apply
    oxe.execute_node(n, execution_context, graph)
  File "/workspace/finn-base/src/finn/core/onnx_exec.py", line 107, in execute_node
    output_list = sess.run(None, input_dict)
  File "/home/pgeel/.local/lib/python3.8/site-packages/onnxruntime/capi/session.py", line 110, in run
    return self._sess.run(output_names, input_feed, run_options)
onnxruntime.capi.onnxruntime_pybind11_state.InvalidArgument: [ONNXRuntimeError] : 2 : INVALID_ARGUMENT : At

Building dataflow accelerator from slice.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in output_files/slice
Build log is at output_files/slice/build_dataflow.log
Running step: step_tidy_up [1/16]
> /home/pgeel/.local/lib/python3.8/site-packages/onnxruntime/capi/session.py(110)run()
    108             output_names = [output.name for output in self._outputs_meta]
    109         try:
--> 110             return self._sess.run(output_names, input_feed, run_options)
    111         except C.EPFail as err:
    112             if self._enable_fallback:

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 1.04 s, sys: 284 ms, total: 1.32 s
Wall time: 1min 36s


-1